In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import h5py


from holodeck import plot, detstats, utils
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC, GYR
import holodeck as holo

import hasasia.sim as hsim

In [ ]:
fobs_cents, fobs_edges = utils.pta_freqs()
sam = holo.sams.Semi_Analytic_Model(shape=30)
hard = holo.hardening.Fixed_Time_2PL_SAM(sam=sam, time=3*GYR)


In [ ]:
hc_ss, hc_bg, sspar, bgpar = sam.gwb(fobs_edges, hard, realize=10, loudest=5, params=True)

In [ ]:
for ii, par in enumerate(bgpar):
    print(f"{sings.par_names[ii]}, {utils.stats(par)}")